In [2]:
# pip install bitsandbytes-cuda12x

In [3]:
# !pip install datasets trl peft -q

In [4]:
# !pip install transformers -q
# !pip install -U bitsandbytes -q

In [5]:
# !pip3 install -q -U bitsandbytes==0.42.0
# !pip3 install -q -U peft==0.8.2
# !pip3 install -q -U trl==0.7.10
# !pip3 install -q -U accelerate==0.27.1
# !pip3 install -q -U datasets==2.17.0
# !pip3 install -q -U transformers==4.38.1
# !pip3 install -q -U optimum

In [6]:
import json
import pandas as pd


from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import setup_chat_format
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer

## Data loading

In [7]:
import torch

torch.cuda.set_device(0) 

In [8]:
with open('2rca_checked_version.json') as f:
    data = json.load(f)

## Data samples

In [9]:
data[0]

{'History': ['Привет, расскажи о себе.',
  'Привет! Под вкусный кофеек настроение поболтать появилось.'],
 'Dia_ID_hash': 'dia_628e79cf',
 'Utt_ID_hash': 'utt_64c12a9b',
 'Phrase': 'Что читаешь? Мне нравится классика. Я тоже люблю пообщаться.',
 'Rewrite': 'Что читаешь? Мне нравится читать классику. Я тоже люблю пообщаться.'}

In [10]:
data[1]

{'History': ['Привет, расскажи о себе.',
  'Привет! Под вкусный кофеек настроение поболтать появилось.',
  'Что читаешь? Мне нравится читать классику. Я тоже люблю пообщаться.'],
 'Dia_ID_hash': 'dia_628e79cf',
 'Utt_ID_hash': 'utt_5cd9dc06',
 'Phrase': 'Люблю животных, просто обожаю, как и свою работу. Я фантастику люблю.',
 'Rewrite': 'Люблю животных, просто обожаю, как и свою работу. Я фантастику читать люблю.'}

In [11]:
data[2]

{'History': ['Привет, расскажи о себе.',
  'Привет! Под вкусный кофеек настроение поболтать появилось.',
  'Что читаешь? Мне нравится читать классику. Я тоже люблю пообщаться.',
  'Люблю животных, просто обожаю, как и свою работу. Я фантастику читать люблю.',
  'А я выращиваю фиалки и веду здоровый и активный образ жизни.'],
 'Dia_ID_hash': 'dia_628e79cf',
 'Utt_ID_hash': 'utt_9d73f6c2',
 'Phrase': 'Ух ты, интересно.',
 'Rewrite': 'Ух ты, интересно, ты фиалки выращиваешь.'}

In [12]:
data[3]

{'History': ['Привет, расскажи о себе.',
  'Привет! Под вкусный кофеек настроение поболтать появилось.',
  'Что читаешь? Мне нравится читать классику. Я тоже люблю пообщаться.',
  'Люблю животных, просто обожаю, как и свою работу. Я фантастику читать люблю.',
  'А я выращиваю фиалки и веду здоровый и активный образ жизни.',
  'Ух ты, интересно, ты фиалки выращиваешь.'],
 'Dia_ID_hash': 'dia_628e79cf',
 'Utt_ID_hash': 'utt_2bfa24e4',
 'Phrase': 'Ты случайно не принц на белом коне? Я его очень жду.',
 'Rewrite': 'Ты случайно не принц на белом коне? Я принца на белом коне очень жду.'}

Saving sample of data for manual examination

In [13]:
history, phrase, rewrite = [], [], []

for sample in data:
    history.append(sample["History"])
    phrase.append(sample["Phrase"])
    rewrite.append(sample["Rewrite"])

In [14]:
df = pd.DataFrame({"history": history, "phrase": phrase, "rewrite": rewrite})

## HF datasets

In [15]:
from datasets import DatasetDict, Dataset

ds = Dataset.from_pandas(df)
ds = ds.train_test_split(test_size=0.2, shuffle=True, seed=42)
train_ds, test_ds = ds["train"], ds["test"]
test_ds = test_ds.train_test_split(test_size=0.5, shuffle=True, seed=42)
val_ds, test_ds = test_ds["train"], test_ds["train"]

ds["train"] = train_ds
ds["val"] = val_ds
ds["test"] = test_ds

In [16]:
ds

DatasetDict({
    train: Dataset({
        features: ['history', 'phrase', 'rewrite'],
        num_rows: 4411
    })
    test: Dataset({
        features: ['history', 'phrase', 'rewrite'],
        num_rows: 551
    })
    val: Dataset({
        features: ['history', 'phrase', 'rewrite'],
        num_rows: 551
    })
})

In [17]:
def process_function(sample):

    prompt = ("Перепиши неполное высказывание на основе истории диалога. Твой ответ должен содержать только переписанное неполное высказвание. "
     + "История: " + sample['history'][-1] #str(sample['history'])
     + " Неполное высказвание: " + sample["phrase"])

    msg = {"prompt": "<start_of_turn>user\n" + prompt,
           "completion": "<start_of_turn>model\n" + sample["rewrite"]}
    return msg

In [18]:
ds['train'] = ds['train'].map(process_function)
ds['val'] = ds['val'].map(process_function)
ds['test'] = ds['test'].map(process_function)

Map:   0%|          | 0/4411 [00:00<?, ? examples/s]

Map:   0%|          | 0/551 [00:00<?, ? examples/s]

Map:   0%|          | 0/551 [00:00<?, ? examples/s]

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "Vikhrmodels/Vikhr-Gemma-2B-instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"": torch.cuda.current_device()}
                                             )

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=1,
    target_modules=["q_proj", "v_proj"],
    task_type="CAUSAL_LM"
)

In [ ]:
ds["train"]["prompt"][0]

'<start_of_turn>user\nПерепиши неполное высказывание на основе истории диалога. Твой ответ должен содержать только переписанное неполное высказвание. История: Моей собаке уже 5 лет, и я даже не представляю, как я могла жить без своей собаки раньше?! Я думаю, что у тебя всё получится и у вас скоро обязательно появится питомец! Ведь собаки такие милые! Что сегодня будешь готовить на ужин? Неполное высказвание: Сегодня будет мясо с кровью! Вот только надо в магазин... Эх, пойду прогуляюсь под дождём, это успокаивает.'

In [ ]:
ds["train"]["completion"][0]

'<start_of_turn>model\nСегодня на ужин будет мясо с кровью! Вот только надо в магазин... Эх, пойду прогуляюсь под дождём, это успокаивает.'

In [ ]:
import random


def seed_everything(seed):
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


seed_everything(42)

In [ ]:
training_args = SFTConfig(packing=True,
                          report_to="wandb",
                          per_device_train_batch_size=1,
                          per_device_eval_batch_size=1,
                          gradient_accumulation_steps=512,
                          num_train_epochs=10,
                          optim="paged_adamw_8bit",
                          learning_rate=3e-03,
                          eos_token="<end_of_turn>",
                          do_eval=True,
                          eval_strategy="steps",
                          eval_steps=1,
                          logging_steps=1)

trainer = SFTTrainer(
    model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["val"],
    peft_config=lora_config
)


trainer.train()

Converting train dataset to ChatML:   0%|          | 0/4411 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/4411 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/4411 [00:00<?, ? examples/s]

Packing train dataset:   0%|          | 0/4411 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/551 [00:00<?, ? examples/s]

...

Tokenizing eval dataset:   0%|          | 0/551 [00:00<?, ? examples/s]

Packing eval dataset:   0%|          | 0/551 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: pvlshkunov to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


It is strongly recommended to train Gemma2 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.


Step,Training Loss,Validation Loss
1,2.043600,1.919052


In [ ]:
messages = [
    {"role": "user", "content": "Перепиши неполное высказывание на основе истории диалога. Твой ответ должен содержать только переписанное неполное высказвание. "
     + "История: " + ds['val']['history'][0][-1]
     + " Неполное высказвание: " + ds['val']["phrase"][0]}
]

In [ ]:
with torch.autocast(device_type='cuda', dtype=torch.float16):
    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to('cuda')
    outputs = model.generate(inputs, max_new_tokens=500, num_beams=2)

In [ ]:
outputs

tensor([[     2,    106,   1645,    108,  38780,   3880,   3843,   2087,  13699,
           7047, 190872,   7788,  16913,   1416,  83511,  63033,  46776,  94040,
         235265,   4987,  10387,  44008,  53287,  53361,   1691,  15757,   7389,
          14867, 172377,   2087,  13699,   7047, 190872, 133999, 235265, 143646,
         235292,  17952,   7540, 235265,  27931,  86342,  17164, 235336,  16339,
          13699,   7047, 190872, 133999, 235292,  13330,   3777,   4987, 104643,
         235269,   2455,  17164, 235336,    107,    108,    106,   2516,    108,
         236038,   3777,   4987, 104643, 235269,   2455,  17164, 235336,    107]],
       device='cuda:0')

In [ ]:
tokenizer.batch_decode(outputs)

['<bos><start_of_turn>user\nПерепиши неполное высказывание на основе истории диалога. Твой ответ должен содержать только переписанное неполное высказвание. История: Привет. Откуда ты? Неполное высказвание: Я из Твери, а ты?<end_of_turn>\n<start_of_turn>model\nЯ из Твери, а ты?<end_of_turn>']

In [ ]:
from tqdm import tqdm

In [ ]:
raw_test_results_new = []
prompt = "Перепиши неполное высказывание на основе истории диалога. Твой ответ должен содержать только переписанное неполное высказвание. "


for i in tqdm(range(len(ds['test']))):

    messages = [
        {
            "role": "user",
            "content": prompt + "История: " + ds['test']['history'][i][-1] + " Неполное высказвание: " + ds['test']["phrase"][i]
            }
        ]
    with torch.autocast(device_type='cuda', dtype=torch.float16):
        inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to('cuda')
        outputs = model.generate(inputs, max_new_tokens=500, num_beams=2)

    out = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    raw_test_results_new.append(out[0])

100%|██████████| 551/551 [1:33:13<00:00, 10.15s/it]  


In [ ]:
raw_test_results_new = [r.split("model\n")[-1] for r in raw_test_results_new]

In [ ]:
import pandas as pd

raw_test_results = pd.DataFrame(raw_test_results_new, columns=['model_out_raw'])

raw_test_results['history'] = ds['test']['history']
raw_test_results['text'] = ds['test']['phrase']
raw_test_results['restored_text'] = ds['test']['rewrite']

raw_test_results.head(10)

,model_out_raw,history,text,restored_text
0,"Я из Твери, а ты?","[Привет., Привет. Откуда ты?]","Я из Твери, а ты?","Я из Твери, а ты откуда?"
1,"Да, именно Питерский политех. Жаль, что не в м...","[Привет., Привет! Сколько тебе лет? Ты где-то ...","Да, именно Питерский. Жаль, что не в месте учи...","Да, именно Питерский политех. Жаль, что не вме..."
2,Мои дети - мои песики. Пока не нашла достойног...,"[Приветики! Как дела?, Привет! Всё хорошо, а у...",Мои дети - мои песики. Пока не нашла достойног...,Мои дети - мои песики. Пока не нашла достойног...
3,Где живёшь? В городе где-то? Какие любишь?,"[Привет., Привет., Расскажи о себе., Я собираю...",Где живёшь? В городе где-то? Какие любишь?,Где живёшь? В городе где-то? Какие фрукты любишь?
4,"Да, это самое главное, с кем ты работаешь?","[Здравствуй. Как тебе погодка?, Привет, я дума...","Да, это самое главное, с кем ты работаешь?","Да, это самое главное что весна придет через п..."
5,"Иногда беру ее с собой, у тебя кто?","[Привет, ты откуда? Я фотограф. Люблю путешест...","Иногда беру ее с собой, у тебя кто?","Иногда беру дочь с собой в путешествие, а у те..."
6,Я всегда за новые знакомства.,"[Добрый вечер, вы не против новых знакомств?]",Я всегда за.,Я всегда за знакомства.
7,Я певица. А вы?,"[Привет., Привет, Алла., Чем вы занимаетесь?]",Певица. А вы?,Я певица. А вы чем занимаетесь?
8,Чтобы не испортить зрение и осанку за компьюте...,"[Привет., О, привет! Рад новому знакомству! Ты...",Чтобы не испортить зрение и осанку за компьюте...,Чтобы не испортить зрение и осанку за компьюте...
9,"Да, и учусь, и работаю.","[Привет., Привет. Как дела?, Отлично у меня де...","Да, и учусь, и работаю.","Да, я и учусь, и работаю."


In [ ]:
MODEL_NAME = "vikhr_gemma_lora"

In [ ]:
raw_test_results.to_csv(f"{MODEL_NAME.split('/')[-1]}_raw_test_results.csv")

In [ ]:
import numpy as np
import sacrebleu
from rouge_metric import PyRouge

rouge = PyRouge(rouge_n=(4), skip_gap=4)


class RestorationFScore:

    def __init__(self, tokenizer, n_gram: int=2):
        self.n_gram = n_gram
        self.tokenizer = tokenizer

    def preprocess(self, sents):
        for sent in sents:
            sent_tokenize = self.tokenizer(sent)['input_ids']
            yield [tuple(sent_tokenize[i:i+self.n_gram]) for i, _ in enumerate(sent_tokenize)]

    def _itereval(self):
        for i, predictions in enumerate(self.predictions):
            restored_ngrams = set(predictions).difference(self.references[i])
            ngrams_in_ref = set(self.rewrites[i]).difference(self.references[i])
            interagree = ngrams_in_ref.intersection(restored_ngrams)
            if len(restored_ngrams):
                precision = len(interagree) / len(restored_ngrams)
            else:
                precision = 0.
            if len(ngrams_in_ref):
                recall = len(interagree) / len(ngrams_in_ref)
            else:
                recall = 0.
            if precision or recall:
                yield 2 * ((precision * recall) / (precision + recall))
            else:
                yield 0.

    def evaluate(self, predictions: list,
                 references: list, rewrites: list):
        self.predictions = [p for p in self.preprocess(predictions)]
        self.references = [p for p in self.preprocess(references)]
        self.rewrites = [p for p in self.preprocess(rewrites)]
        return np.mean(list(self._itereval()))

In [ ]:
def callculate_metrics(row):
    row["bleu_score"] = sacrebleu.corpus_bleu(row.model_out_raw, [row.text]).score
    rouge_scores = rouge.evaluate(row.model_out_raw,
                                  [[t] for t in row.text])
    for k in rouge_scores:
        row[k] = rouge_scores[k]['f']

    for n in range(1, 5):
        rf_score = RestorationFScore(tokenizer, n)
        row[f"rf_score_{n}"] = rf_score.evaluate(predictions=row.model_out_raw,
                                                 references=row.text,
                                                 rewrites=row.restored_text)
    return row

In [ ]:
raw_test_results["type"] = "2rca"
raw_test_results = raw_test_results.groupby(by="type").agg(list)

In [ ]:
raw_test_results.apply(callculate_metrics, axis=1).drop(columns=["model_out_raw", "history", "text", "restored_text"])

,bleu_score,rouge-1,rouge-2,rouge-3,rouge-4,rouge-l,rf_score_1,rf_score_2,rf_score_3,rf_score_4
type,,,,,,,,,,
2rca,81.584098,0.827842,0.773599,0.734291,0.691708,0.827763,0.129307,0.102482,0.090379,0.0842
